In [90]:
import pandas as pd
import numpy as np

In [91]:
import pandas as pd
import numpy as np

In [92]:
# def fix_address(address):
#     return (address.split(sep=', '))[0]

In [93]:
# conda install -c conda-forge geopy
from geopy.geocoders import Nominatim
from geopy.distance import distance
from tqdm import tqdm

# proceed with cauction!!!
def distance_from(given_location, dataframe, addresses_column_name, postifx="", inplace=False):
    """
    dataframe - pandas.DataFrame object
    addresses_column_name - name of the column in which we can find addresses
    given_location - tuple with latitude and longitude of a location from which we're counting the distance

    postfix - what should I add to every address to make sure it's what we need. example: city/country
    inplace - check it if you want to change the given dataframe. default False

    returned value - pd.Series distances in kilometers
    """
    geolocator = Nominatim(user_agent="Strive School YMVC Project")

    number_of_rows = dataframe.shape[0]
    addresses_df = dataframe[addresses_column_name]

    gps_addresses = np.zeros(shape=number_of_rows)

    # this loop can take a looooong time.
    for row_nr in np.arange(0, number_of_rows):
        if row_nr % 20 == 0:
            print(row_nr, number_of_rows)
        dist = np.nan
        try:
            loc = geolocator.geocode(addresses_df.iloc[row_nr] + postifx)
            dist = distance(given_location, (loc.latitude, loc.longitude)).kilometers
        except Exception as e:
            print(e)
        gps_addresses[row_nr] = dist

    if inplace:
        # we put a new column before the "address" column
        dataframe.insert(dataframe.columns.get_loc(addresses_column_name), 'distance_km', gps_addresses)
    else:
        return pd.Series(gps_addresses, name='distance_km')

In [94]:
def contains(word, column, ignore_case=False):
    """
    Usage:
    Pass a column and a word u want to find in the cells, to get a np.array of type bool.
    Afterwards u can pass it as an argument of [] selection mechanism. Examples:

    austin_tx[ contains(austin_tx.categories, "Shopping") ]
    austin_tx[ contains(austin_tx.categories, "Shopping") | contains(austin_tx.categories, "Hotels") ]      # and
    austin_tx[ contains(austin_tx.categories, "Shopping") & contains(austin_tx.categories, "Watches") ]     # or

    Works for all columns that return strings. hours and attributes too.
    """
    if(ignore_case):
        return np.array([(word.lower() in vals.lower()) for vals in column])
    else:
        return np.array([(word in vals) for vals in column])

# CONTAINS ANY

In [95]:
def contains_any(words_list, column, ignore_case=False):
    contains = np.zeros(len(column), dtype=bool)
    for idx, string in enumerate(column):
        for w in words_list:
            if ignore_case:
                if w.lower() in string.lower():
                    contains[idx] = True
                    break
            else:
                if w in string:
                    contains[idx] = True
                    break
            
    return contains

In [96]:
# def fix_category(category):
#     return category.replace(", &", " &")

In [97]:
def get_categories_dict(category_column, separator):
    categories_counts = {}
    for things in category_column:
        for thing in things.split(sep=separator):    
            if thing in categories_counts.keys():
                categories_counts[thing] += 1
            else:
                categories_counts[thing] = 1
    return categories_counts

In [98]:
berlin = pd.read_csv('berlin_df.csv')

In [99]:
# berlin.drop(['url', 'claimed'], axis=1, inplace=True)

In [100]:
# print(berlin.columns.values)

In [101]:
# berlin.isna().sum()

In [102]:
# berlin.drop(berlin.index[berlin['categories'].isna()], inplace=True)


In [103]:
# berlin.isna().sum()

In [104]:
# import re
def fix_names(dataframe, column_name):
    dataframe[column_name] = dataframe[column_name].apply(lambda name: e.sub(r"(\w)([A-Z])", r"\1 \2", name))

In [105]:
# fix_names(berlin, 'Restaurants_Name')

In [106]:
# berlin.shape[0]

In [107]:
# graph stars vs distance

In [108]:
# distance_from((52.525579992852855, 13.369230338630214),berlin, 'address',  inplace=True)


In [109]:
# berlin.dropna(inplace=True)

# FUCKING HERE

In [110]:
def set_category(cat_name, dataframe, new_column_name, bool_table):
    for idx in range(dataframe.shape[0]):
        if bool_table[idx]:
            dataframe[new_column_name].iloc[idx] = cat_name


In [111]:
restaurants = ["reservations" ,"delivery", "burgers", "chinese", "japanese", "mexican", "italian",  "thai", "japanese", "mexican", "italian", 'sushi', 'ramen', 'kebab', 'coffee', 'tea', 'falafel', 'cafes', 'Vietnamese', 'German']
restaurants += ['Mediterranean', 'African', 'Food', 'breakfast', 'cake', 'lebanese', 'pizza', 'greek','European', 'vegan', 'Korean', 'Dim Sum', 'Oriental','Moroccan','Bakeries', 'indian','american','Ethiopian', 'Arabian', 'Ice Cream']
restaurants += ['dessert', 'yoghurt', 'austrian', 'Crepes', 'pub', 'gastro', 'brewer', 'Spanish', 'French', 'Brasserie', 'bar']
auto_services = ['autorepair', 'car_dealers', 'auto_detailing']
others = ['dryclean', 'hair', 'mobilephonerepair', 'gyms', 'bars', 'massage', 'nightlife', 'shopping', 'museum', 'forest', 'historic building', 'aquarium', 'zoo', 'landmark']
is_restaurant = contains_any(restaurants, berlin['categories'], ignore_case=True)
is_auto_service = contains_any(auto_services, berlin['categories'], ignore_case=True)
is_others = contains_any(others, berlin['categories'], ignore_case=True)

In [112]:
berlin['main_category'] = np.nan
set_category('Other', berlin, 'main_category', is_others)
set_category('Restaurant', berlin, 'main_category', is_restaurant)
set_category('Auto Service', berlin, 'main_category', is_auto_service)
berlin[berlin['main_category'].isna()].shape[0]

C:\my\dev\tools\anaconda3\envs\numba\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


34

In [113]:
# main_categories = get_categories_dict(berlin.main_category, separator=", ")

In [114]:
# main_categories

In [115]:
restaurant_categories = get_categories_dict(berlin[berlin.main_category=='Restaurant'].categories, ',')            # dictionary of categories
resturant_cat_df = pd.DataFrame([restaurant_categories.keys(), restaurant_categories.values()]).T   # dataframe out of that dictionary
resturant_cat_df.rename(columns={0:'category',1:'count'}, inplace=True)       # name columns in a fasionable name

# RESTAURANTS

In [116]:
avg_stars=[]
avg_rev_count=[]
for category in resturant_cat_df['category']:
    avg_stars.append(berlin[contains(category, berlin.categories)]['stars'].mean())
    avg_rev_count.append(berlin[contains(category, berlin.categories)]['review_count'].mean())

resturant_cat_df['average_stars'] = avg_stars
resturant_cat_df['average_review_count'] = avg_rev_count

In [117]:
resturant_cat_df.sort_values(by='count', ascending=False)

,category,count,average_stars,average_review_count
17,Italian,184,4.255435,37.038043
51,Thai,150,3.846667,27.253333
57,Vietnamese,67,4.007463,54.298507
42,Mexican,66,3.356061,38.484848
15,Pizza,64,4.132812,51.484375
...,...,...,...,...
92,International Grocery,1,4.500000,13.000000
93,Beach Bars,1,3.000000,62.000000
94,Diners,1,4.000000,19.000000
27,Fish & Chips,1,4.500000,116.000000


In [118]:
(berlin['main_category'] == 'Restaurant').sum()

755

In [119]:
resturant_cat_df.to_csv("all_rest_cat.csv", index=False)

In [120]:
resturant_cat_df['average_review_count'] > 50.0
shitty_businesessss = resturant_cat_df[(resturant_cat_df['average_stars'] <= 3.5) & (resturant_cat_df['average_review_count'] > 35.0)].sort_values(by=['average_stars', 'count'], ascending=[True, False])

In [121]:
shitty_businesessss.to_csv("shit.csv", index=False) 

In [122]:
resturant_cat_df[resturant_cat_df['average_stars'] < 4].sort_values(by=['count','average_stars'], ascending=[False, True])

,category,count,average_stars,average_review_count
51,Thai,150,3.846667,27.253333
42,Mexican,66,3.356061,38.484848
66,Sushi,39,3.923077,42.923077
47,Pan Asian,35,3.961538,49.974359
22,Chinese,27,3.925926,50.222222
34,Cocktail Bars,25,3.700000,42.400000
11,Takeaway & Fast Food,18,3.888889,121.444444
43,Steakhouses,17,3.705882,98.823529
9,Ramen,15,3.933333,34.200000
38,BBQ & Barbecue,13,3.884615,70.153846


# OTHER

In [123]:
other_categories = get_categories_dict(berlin[berlin.main_category=='Other'].categories, ',')
other_cat_df = pd.DataFrame([other_categories.keys(), other_categories.values()]).T
other_cat_df.rename(columns={0:'category',1:'count'}, inplace=True)

In [124]:
avg_stars=[]
avg_rev_count=[]
for category in other_cat_df['category']:
    avg_stars.append(berlin[contains(category, berlin.categories)]['stars'].mean())
    avg_rev_count.append(berlin[contains(category, berlin.categories)]['review_count'].mean())

other_cat_df['average_stars'] = avg_stars
other_cat_df['average_review_count'] = avg_rev_count

In [129]:
other = other_cat_df.sort_values(by='average_stars', ascending=True)
other.to_csv("other.csv", index=False)

In [126]:
categories = get_categories_dict(berlin.categories, ',')            # dictionary of categories
cat_df = pd.DataFrame([categories.keys(), categories.values()]).T   # dataframe out of that dictionary
cat_df.rename(columns={0:'category',1:'count'}, inplace=True)       # name columns in a fasionable name

In [127]:
avg_stars=[]
avg_rev_count=[]
for category in cat_df['category']:
    avg_stars.append(berlin[contains(category, berlin.categories)]['stars'].mean())
    avg_rev_count.append(berlin[contains(category, berlin.categories)]['review_count'].mean())

cat_df['average_stars'] = avg_stars
cat_df['average_review_count'] = avg_rev_count

In [128]:
size_cat_df = cat_df.shape[0]
twenty_percent = size_cat_df // 5

cat_df.sort_values(by='average_stars', ascending=True).head(twenty_percent).sort_values(by='average_review_count', ascending=False).head(twenty_percent//5)

,category,count,average_stars,average_review_count
137,Cafeterias,1,3.500000,203.000000
84,Baden,1,4.000000,174.000000
100,Bavarian,1,3.500000,147.000000
49,Dive Bars,5,4.000000,130.200000
11,Takeaway & Fast Food,18,3.888889,121.444444
47,Steakhouses,17,3.705882,98.823529


In [132]:
ber_address = berlin.dropna()

In [135]:
(ber_address.corr()).to_csv("insignificant_correlations.csv")